
When it comes to partitioning of the data, the following points should be considered:<br>
* parallel processing has to be possible
* most of the time, the all the data is processed as one group
* it should also be possible to read data for a single company in an efficient way

As a simple approach, we will create new month and year columns for the "period" column and we will create a cik_mod_10 column, which is the cik module 10.<br>
This gives us the possibility to create the partition as year-month-cik_mod_10. Since we have about 10 years of data, this will create about 10*12*10=1200 partitions. So, every partition should have about 110'000'000 / 1'200, which is approx 100'000 entries.<br>
This way, we should be able to select efficiently by the period date and by the cik number (provided that we provide the appropriate selects criterias.

### Create partitions with the test dataframe

In [ ]:
from pyspark.sql.functions import col, year, month

df_tst_partition = df_tst_join.withColumn("period_year", year("period")) \
                         .withColumn("period_month", month("period")) \
                         .withColumn("cik_mod_10", col("cik") % 5)

In [ ]:
shutil.rmtree(tst_parquet_folder,  ignore_errors=True)
start = time.time()
df_tst_partition.write.partitionBy('period_year', 'period_month', 'cik_mod_10').parquet(tst_parquet_folder)
duration = time.time() - start
print("duration: ", duration)

duration:  338.159051656723


In [ ]:
result = df_tst_partition.filter(df_tst_partition.period_year == 2011)

In [ ]:
result.show(2)

+-------+--------------------+--------------------+------------+-----+----------+----+------+-----+--------+------------------+----+---------+------+---------------+-----+--------------------+----+------------+---------+------+---------------+-----+--------------------+----+----------+-------+---+--------------------+--------+-----+----+----+----+----------+----+---+----------+--------------------+-------+------+-----------------+-----+-----+------+----+----+-----+-----+--------------------+--------+------+--------------------+--------+-----------+------------+----------+
|    cik|                adsh|                 tag|     version|coreg|     ddate|qtrs|   uom|value|footnote|              name| sic|countryba|stprba|         cityba|zipba|                bas1|bas2|        baph|countryma|stprma|         cityma|zipma|                mas1|mas2|countryinc|stprinc|ein|              former| changed|  afs|wksi| fye|form|    period|  fy| fp|     filed|            accepted|prevrpt|detail|     